In [1]:
import os
import numpy as np
import torch
import torch.nn.functional as F

from detection_utils import random_label

# from utils import transform
import torch.optim as optim

import torchvision
from torchvision import transforms
import torch.utils.data as data

from tqdm.notebook import tqdm
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import json

In [2]:
model = torchvision.models.resnet50(pretrained=True)

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "/Users/albertwen/Downloads/mae_data/resnet50.pth"

# Setting the seed
# pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

pretrained_model = torchvision.models.resnet50(pretrained=True)
pretrained_model = pretrained_model.to(device)

if CHECKPOINT_PATH:
    if os.path.isfile(CHECKPOINT_PATH):
        checkpoint = torch.load(CHECKPOINT_PATH) 
        # model.load_state_dict(checkpoint['state_dict'])  #['state_dict']
        pretrained_model.load_state_dict(checkpoint)  #['state_dict']
        print("=> load chechpoint found at {}".format(CHECKPOINT_PATH))
        # print("=> loaded checkpoint '{}' (epoch {})"
        #       .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(CHECKPOINT_PATH))
# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

Using device cpu
=> load chechpoint found at /Users/albertwen/Downloads/mae_data/resnet50.pth


In [3]:
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256), transforms.CenterCrop(224),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
])

In [4]:
imagenet_path = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/"
val_dataset = torchvision.datasets.ImageFolder(root=os.path.join(imagenet_path, 'val'), transform=plain_transforms)
val_dataset_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=8)

In [5]:
# preds = []
# reals = []

# for images, labels in (val_dataset_loader):
#     if labels.data != 0:
#         break
#     preds.append(model(images).data.max(1, keepdim=True)[1][0])
#     reals.append(labels.data)    

In [6]:
# images.shape

In [7]:
# pretrained_model(x_var.clone()).data.max(1, keepdim=True)[1][0].item()

In [8]:
imagenet_path = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/"
val_dataset = torchvision.datasets.ImageFolder(root=os.path.join(imagenet_path, "test_val3"), transform=plain_transforms)
val_dataset_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=8)

In [9]:
labels = []
for images, label in val_dataset_loader:
    labels.append(label.to(device))

In [11]:
sum(labels)

tensor([0])

In [11]:
labels

[tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tenso

In [79]:
def untargeted_detection(model, img, true_label, dataset, lr, u_radius, cap=1000, margin=20, use_margin=False):
    model.eval()
    # x_var = torch.autograd.Variable(img.clone().cuda(), requires_grad=True)
    x_var = torch.autograd.Variable(img.clone(), requires_grad=True)
    # true_label = model(x_var.clone()).data.max(1, keepdim=True)[1][0].item()
    optimizer_s = optim.SGD([x_var], lr=lr)
    counter = 0
    while model(x_var.clone()).data.max(1, keepdim=True)[1][0].item() == true_label:
        optimizer_s.zero_grad()
        output = model(x_var)
        if use_margin:
            _, top2_1 = output.data.cpu().topk(2)
            argmaxl1 = top2_1[0][0]
            if argmaxl1 == true_label:
                argmaxl1 = top2_1[0][1]
            loss = (output[0][true_label] - output[0][argmaxl1] + margin).clamp(min=0)
        else: 
            # loss = -F.cross_entropy(output, torch.LongTensor([true_label]).cuda())
            loss = -F.cross_entropy(output, torch.LongTensor([true_label]))
        loss.backward()

        x_var.data = torch.clamp(x_var - lr * x_var.grad.data, min=0, max=1)
        x_var.data = torch.clamp(x_var - img, min=-u_radius, max=u_radius) + img
        counter += 1
        if counter >= cap:
            break
    return counter

In [80]:
def untargeted_vals(model, dataset, attack, real_dir, adv_dir, untargeted_lr, u_radius):
    vals = np.zeros(0)
    if attack == "real":

        image_dir = os.path.join(real_dir, 'val')
        assert os.path.exists(image_dir)
        val_dataset = torchvision.datasets.ImageFolder(root=image_dir, transform=plain_transforms)
        val_dataset_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)
        for img, label in (val_dataset_loader):

        # for i in range(lowind, upind):
            # image_dir = os.path.join(real_dir, str(i) + '_img.pt')
            # assert os.path.exists(image_dir)
            # view_data = torch.load(image_dir)

            model.eval()
            val = untargeted_detection(model, img, label, dataset, untargeted_lr, u_radius)
            vals = np.concatenate((vals, [val]))
    else: 
        cout = 0
        image_dir = os.path.join(os.path.join(adv_dir, "val"), attack)
        assert os.path.exists(image_dir)
        atk_dataset = torchvision.datasets.ImageFolder(root=image_dir, transform=plain_transforms)
        atk_dataset_loader = data.DataLoader(atk_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)
        for img, label in (atk_dataset_loader):

        # for i in range(lowind, upind):
            # image_dir = os.path.join(os.path.join(adv_dir, attack), str(i) + title + '.pt')
            # assert os.path.exists(image_dir)
            # adv = torch.load(image_dir)
            # real_label = torch.load(os.path.join(real_dir, str(i) + "_label.pt")) # TODO: edit
            model.eval()
            # predicted_label = model(transform(adv.clone(), dataset=dataset)).data.max(1, keepdim=True)[1][0]
            predicted_label = model(img).data.max(1, keepdim=True)[1][0]
            real_label = label
            if real_label == predicted_label:
                cout -= 1
                continue
            val = untargeted_detection(model, img, dataset, untargeted_lr, u_radius)
            vals = np.concatenate((vals, [val]))
        print('this is the number of successes in untargeted detection', cout)
    return vals

In [81]:
def targeted_detection(model, img, true_label, dataset, lr, t_radius, cap=200, margin=20, use_margin=False):
    model.eval()
    # x_var = torch.autograd.Variable(img.clone().cuda(), requires_grad=True
    x_var = torch.autograd.Variable(img.clone(), requires_grad=True)
    # true_label = model(transform(x_var.clone(), dataset=dataset)).data.max(1, keepdim=True)[1][0].item()
    # change this function to pass in the true label

    optimizer_s = optim.SGD([x_var], lr=lr)
    # target_l = torch.LongTensor([random_label(true_label, dataset=dataset)]).cuda()
    target_l = torch.LongTensor([random_label(true_label, dataset=dataset)])
    counter = 0

    # while model(transform(x_var.clone(), dataset=dataset)).data.max(1, keepdim=True)[1][0].item() == true_label: 
    while model(x_var.clone()).data.max(1, keepdim=True)[1][0].item() == true_label:
        optimizer_s.zero_grad()
        # output = model(transform(x_var, dataset=dataset))
        output = model(x_var)
        if use_margin:
            target_l = target_l[0].item()
            _, top2_1 = output.data.cpu().topk(2)
            argmaxl1 = top2_1[0][0]
            if argmaxl1 == target_l:
                argmaxl1 = top2_1[0][1]
            loss = (output[0][argmaxl1] - output[0][target_l] + margin).clamp(min=0)
        else:
            loss = F.cross_entropy(output, target_l)
        loss.backward()

        x_var.data = torch.clamp(x_var - lr * x_var.grad.data, min=0, max=1)
        x_var.data = torch.clamp(x_var - img, min=-t_radius, max=t_radius) + img
        counter += 1
        if counter >= cap:
            break
    return counter

In [82]:
def targeted_vals(model, dataset, attack, real_dir, adv_dir, targeted_lr, t_radius):
    vals = np.zeros(0)
    if attack == "real":
        # for i in range(lowind, upind):

        # imagenet_path = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/"
        image_dir = real_dir
        assert os.path.exists(image_dir)
        val_dataset = torchvision.datasets.ImageFolder(root=image_dir, transform=plain_transforms)
        val_dataset_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)

        iters = 0
        for img, label in tqdm(val_dataset_loader, desc="Validating..."):
            # image_dir = os.path.join(real_dir, str(i) + "_img.pt") # TODO: edit
            # assert os.path.exists(image_dir)
        
            model.eval()
            # print("targeted vals img", img)
            # print("targeted vals label", label)
            predicted_label = model(img.clone()).data.max(1, keepdim=True)[1][0]
            if label != predicted_label:
                continue

            val = targeted_detection(model, img, label, dataset, targeted_lr, t_radius)
            vals = np.concatenate((vals, [val]))
            iters += 1
            if iters > 5:
                break

    else: 
        cout = 0 # TODO: edit if statement logic

        iters = 0
        image_dir = adv_dir
        assert os.path.exists(image_dir)
        atk_dataset = torchvision.datasets.ImageFolder(root=image_dir, transform=plain_transforms)
        atk_dataset_loader = data.DataLoader(atk_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)

        for img, label in tqdm(atk_dataset_loader, desc="Validating..."):
        # for i in range(lowind, upind):
            # image_dir = os.path.join(os.path.join(adv_dir, attack), str(i) + title + '.pt')
            # assert os.path.exists(image_dir)
            # adv = torch.load(image_dir)
            # real_label = torch.load(os.path.join(real_dir, str(i) + '_label.pt'))
            model.eval()
            predicted_label = model(img).data.max(1, keepdim=True)[1][0]
            real_label = label
            if real_label == predicted_label: # TODO: edit
                cout -= 1
                continue
            iters += 1
            if iters > 5:
                break
            val = targeted_detection(model, img, label, dataset, targeted_lr, t_radius)
            vals = np.concatenate((vals, [val]))
            
        print('this is the number of successes in targeted detection', cout)
    return vals

In [86]:
real_dir = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/val"
adv_dir = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/attack/val"

values = targeted_vals(pretrained_model, "imagenet", "real", real_dir, adv_dir, targeted_lr=0.005, t_radius=0.03)

Validating...:   0%|          | 0/100000 [00:00<?, ?it/s]

In [87]:
values

array([200.,  77., 118., 200., 200., 200.])

In [88]:
targeted_atk_vals = targeted_vals(pretrained_model, "imagenet", "fgsm_4", real_dir, adv_dir, targeted_lr=0.005, t_radius=0.03)

Validating...:   0%|          | 0/400284 [00:00<?, ?it/s]

this is the number of successes in targeted detection -46


In [89]:
targeted_atk_vals

array([0., 0., 0., 0., 0.])

In [39]:
untargeted_values = untargeted_vals(pretrained_model, "imagenet", "real", real_dir, adv_dir, untargeted_lr=0.1, u_radius=0.03)

KeyboardInterrupt: 

NameError: name 'untargeted_values' is not defined